# Ethan's notebook
Used for building additional visuals

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from helper_code import *
print("import successful")

import successful


In [2]:
from eng_helper_code import *
print("import successful")

import successful


# Part 1 : Loading in Datasets

# Load National Survey on Drug Use and Health Datasets

In [ ]:
NSDUH2018df = buildNSDUH2018df()
NSDUH2019df = buildNSDUH2019df()
NSDUH2021df = buildNSDUH2021df()
print(NSDUH2018df.head())
print(NSDUH2019df.head())
print(NSDUH2021df.head())

# Load Core Trends Datasets

In [ ]:
coreTrends2018df = buildCoreTrends2018df()
coreTrends2019df = buildCoreTrends2019df()
coreTrends2021df = buildCoreTrends2021df()
print(coreTrends2018df.head())
print(coreTrends2019df.head())
print(coreTrends2021df.head())

# Part 2 : Visual Graphs

# Graph 1: People by age and frequency of social media usage
This first shows people by age, and their frequency of social media usage.\
Each line shows the data from each year

In [ ]:
def snsPlot(df, label, ax, color='blue', linestyle='--'):
    # Calculate the sum of all columns starting with 'Sns'
    sns_columns = df.filter(like='Sns_').columns
    sns_columns = sns_columns.drop('Sns_YouTube')  # Exclude Sns_YouTube from the analysis because it's questionable social media status
    df['sns_sum'] = df[sns_columns].sum(axis=1, min_count=1)  # min_count=1 ensures NaN if all are NaN

    # Define age groups
    df['age_group'] = pd.cut(df['age'], bins=[0, 25, 34, 49, 64, float('inf')], labels=['18-25', '26-34', '35-49', '50-64', '65+'])
    
    # Determine the number of people with SNS sum >= 5.0 in each age group
    high_sns = df[df['sns_sum'] >= 5.0].groupby('age_group').size().reset_index(name='high_sns_count')

    # Calculate total people in each age group
    total_counts = df.groupby('age_group').size().reset_index(name='total')

    # Merge to get total counts in the high_sns DataFrame
    high_sns = pd.merge(high_sns, total_counts, on='age_group', how='right')

    # Fill NaN with 0 where there are no counts
    high_sns['high_sns_count'] = high_sns['high_sns_count'].fillna(0)

    # Calculate the percentage
    high_sns['percentage'] = (high_sns['high_sns_count'] / high_sns['total']) * 100

    # Optional: Apply a moving average for smoothing
    high_sns['smoothed_percentage'] = high_sns['percentage'].rolling(window=3, min_periods=1).mean()

    # Plot using seaborn on the given axis object
    sns.lineplot(data=high_sns, x='age_group', y='smoothed_percentage', ax=ax, label=f"High Social Media Usage {label}", color=color, linestyle=linestyle)

# Create a single figure and axis for all plots
fig, ax = plt.subplots(figsize=(10, 6))

# Call snsPlot for each DataFrame and year
snsPlot(coreTrends2018df, '2018', ax, 'blue')
snsPlot(coreTrends2019df, '2019', ax, 'orange')
snsPlot(coreTrends2021df, '2021', ax, 'green')

# Set titles and labels
sns.set_palette("dark")
plt.title('Percentage of People with High Social Media Usage by Age Group Across Years')
plt.xlabel('Age Group')
plt.ylabel('Percentage of People')
plt.legend()

# Show the plot
plt.show()

# Graph 2: People by Age and Mental health that affects social activities 
This second graph shows the percentage of people by age that have mental health issues that affect their social activities

In [ ]:
def diffSocialPlot(df, ax, label, color='blue', linestyle='-'):
    # Determine if diff_social is 3 or higher
    df['diff_social_high'] = (df['diff_social']) >= 3

    # Group by age and count how many meet the criteria
    group_counts = df.groupby('age')['diff_social_high'].sum()

    # Calculate total people in each age group
    total_counts = df.groupby('age').size()

    # Calculate the percentage
    percentages = (group_counts / total_counts) * 100

    # Create a DataFrame for plotting
    plot_data = pd.DataFrame({
        'Age Group': percentages.index,
        'Percentage': percentages.values
    })

    # Calculate moving average for smoothing (e.g., over 3 data points)
    plot_data['Moving Avg'] = plot_data['Percentage'].rolling(window=3, min_periods=1).mean()

    # Plot using seaborn with optional smoothing on the given axis
    sns.lineplot(data=plot_data, x='Age Group', y='Moving Avg', ax=ax, label=f"Social Life impacted by Mental Health {label}", color=color, linestyle=linestyle)

# Create a single figure and axis for all plots
fig, ax = plt.subplots(figsize=(10, 6))

# Call diffSocialPlot for each DataFrame and year, using the same axis
diffSocialPlot(NSDUH2018df, ax, '2018', 'blue')
diffSocialPlot(NSDUH2019df, ax, '2019', 'orange')
diffSocialPlot(NSDUH2021df, ax, '2021', 'green')

# Set the title and labels, and show the legend
sns.set_palette("dark")
plt.title('Percentage of People whose Social Life is impacted by their Mental Health by Age Group Across Years')
plt.xlabel('Age Group')
plt.ylabel('Percentage of People')
plt.legend(title='Year')

# Show the plot
plt.show()

# Graph 3: Combining Graphs 1 and 2 to show a Correlation between Social Media Usage and Mental Health affecting Social Activities

In [ ]:
# Create a single figure and axis for all plots
fig, ax = plt.subplots(figsize=(10, 6))

# Call snsPlot for each DataFrame and year
snsPlot(coreTrends2018df, '2018', ax, 'blue')
snsPlot(coreTrends2019df, '2019', ax, 'orange')
snsPlot(coreTrends2021df, '2021', ax, 'green')

# Call diffSocialPlot for each DataFrame and year, using the same axis
diffSocialPlot(NSDUH2018df, ax, '2018', 'blue')
diffSocialPlot(NSDUH2019df, ax, '2019', 'orange')
diffSocialPlot(NSDUH2021df, ax, '2021', 'green')

# Set the title and labels, and show the legend
sns.set_palette("dark")
plt.title('Percentage of People whose Social Life is impacted by Mental Health against Social Media Use')
plt.xlabel('Age Group')
plt.ylabel('Percentage of People')
plt.legend(title='Year')

# Show the plot
plt.show()

# Graph 4 : People by and Age and Feelings of Worthlessness

In [ ]:
def worthlessPlot(df, ax, label, color='blue', linestyle='-'):
    # Determine if worse_worthless is 3 or higher
    df['worthless_high'] = df['worse_worthless'] >= 3

    # Group by age and count how many meet the criteria
    group_counts = df.groupby('age')['worthless_high'].sum()

    # Calculate total people in each age group
    total_counts = df.groupby('age').size()

    # Calculate the percentage
    percentages = (group_counts / total_counts) * 100

    # Create a DataFrame for plotting
    plot_data = pd.DataFrame({
        'Age Group': percentages.index,
        'Percentage': percentages.values
    })

    # Calculate moving average for smoothing (e.g., over 3 data points)
    plot_data['Moving Avg'] = plot_data['Percentage'].rolling(window=3, min_periods=1).mean()

    # Plot using seaborn with optional smoothing on the given axis
    sns.lineplot(data=plot_data, x='Age Group', y='Moving Avg', ax=ax, label=f"Feelings of Worthlessness {label}", color=color, linestyle=linestyle)

# Create a single figure and axis for all plots
fig, ax = plt.subplots(figsize=(10, 6))

# Call snsPlot for each DataFrame and year
snsPlot(coreTrends2018df, '2018', ax, 'green')
snsPlot(coreTrends2019df, '2019', ax, 'red')
snsPlot(coreTrends2021df, '2021', ax, 'blue')

# Call diffSocialPlot for each DataFrame and year, using the same axis
worthlessPlot(NSDUH2018df, ax, '2018', 'green', '-')
worthlessPlot(NSDUH2019df, ax, '2019', 'red', '-')
worthlessPlot(NSDUH2021df, ax, '2021', 'blue', '-')

# Set the title and labels, and show the legend
plt.title('Percentage of People who have had feelings of worthlessness by Age Group Across Years')
plt.xlabel('Age Group')
plt.ylabel('Percentage of People')
plt.legend(title='Year')

# Show the plot
plt.show()

# Graph Alternatives

First combining all data sets, with a column which states the year

In [ ]:
coreTrends2018df['year'] = "2018"
coreTrends2019df['year'] = "2019"
coreTrends2021df['year'] = "2021"

coreTrendsAlldf = pd.concat([coreTrends2018df, coreTrends2019df, coreTrends2021df])
print(coreTrendsAlldf.head())

In [ ]:
NSDUH2018df['year'] = "2018"
NSDUH2019df['year'] = "2019"
NSDUH2021df['year'] = "2021"

NSDUHAlldf = pd.concat([NSDUH2018df, NSDUH2019df, NSDUH2021df])
print(NSDUHAlldf.head())

# Graph 1b

In [ ]:
def snsAllPlot(df, window=3):
    # Calculating sns_sum only if it is not already calculated in your data
    if 'sns_sum' not in df.columns:
        sns_columns = df.filter(like='Sns_').columns
        sns_columns = sns_columns.drop('Sns_YouTube')
        df['sns_sum'] = df[sns_columns].sum(axis=1)

    # Grouping by year and age_group
    grouped = df.groupby(['year', 'age_group'])

    # Calculating the percentage of people with sns_sum >= 5
    percentage_df = grouped.apply(lambda x: (x['sns_sum'] >= 5).mean() * 100).reset_index()
    percentage_df.columns = ['year', 'age_group', 'percentage']

    # Sorting the data to ensure the rolling mean is applied correctly
    percentage_df.sort_values(by=['age_group', 'year'], inplace=True)

    # Calculating the moving average
    percentage_df['smoothed_percentage'] = percentage_df.groupby('age_group')['percentage'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)

    # Plotting using the smoothed data
    sns.lineplot(x='year', y='smoothed_percentage', hue='age_group', data=percentage_df)
    plt.title('Smoothed Percentage of People with SNS Sum ≥ 5 by Year and Age Group')
    plt.xlabel('Year')
    plt.ylabel('Smoothed Percentage')
    plt.legend(title='Age Group')
    plt.show()

    
snsAllPlot(coreTrendsAlldf)

# Graph 2b

In [ ]:
def diffSocialAllPlot(df):
    # Define a new column to check if diff_social is 3 or higher
    df['diff_social_high'] = df['diff_social'] >= 3

    # Group by year and age, calculate total and number meeting criteria
    grouped = df.groupby(['year', 'age']).agg(
        total=('diff_social_high', 'size'),
        high_social_count=('diff_social_high', 'sum')
    ).reset_index()

    # Calculate percentage
    grouped['percentage'] = (grouped['high_social_count'] / grouped['total']) * 100

    # Plotting
    plt.figure(figsize=(12, 8))
    sns.lineplot(data=grouped, x='year', y='percentage', hue='age')
    plt.title('Percentage of People with Diff_Social >= 3 by Year and Age Group')
    plt.xlabel('Year')
    plt.ylabel('Percentage with Diff_Social >= 3')
    plt.legend(title='Age Group')

    # Show the plot
    plt.show()

    
diffSocialAllPlot(NSDUHAlldf)

# Graph 5 : Suicidal Thoughts 

The goal of this graph is to find a way of portraying the relationship between Suicidal Thoughts and Social Media use

In [ ]:
# Rebuild Fresh Datasets
coreTrends2018df = buildCoreTrends2018df()
coreTrends2019df = buildCoreTrends2019df()
coreTrends2021df = buildCoreTrends2021df()

NSDUH2018df = buildNSDUH2018df()
NSDUH2019df = buildNSDUH2019df()
NSDUH2021df = buildNSDUH2021df()

# Merge Datasets
coreTrends2018df['year'] = "2018"
coreTrends2019df['year'] = "2019"
coreTrends2021df['year'] = "2021"

NSDUH2018df['year'] = "2018"
NSDUH2019df['year'] = "2019"
NSDUH2021df['year'] = "2021"

NSDUHAlldf = pd.concat([NSDUH2018df, NSDUH2019df, NSDUH2021df])
print(NSDUHAlldf.head())

coreTrendsAlldf = pd.concat([coreTrends2018df, coreTrends2019df, coreTrends2021df])
print(coreTrendsAlldf.head())


In [ ]:
# Build Dataset for Suicidal Thoughts
suicidalThoughtsdf = NSDUHAlldf.filter(['age', 'year', 'suicidal_think'])

# only have rows where they answered yes
suicidalThoughtsdf = suicidalThoughtsdf[suicidalThoughtsdf['suicidal_think'] == 1]

# drop 65 + cause there are none
suicidalThoughtsdf = suicidalThoughtsdf[suicidalThoughtsdf['age'] != '65+']
count_data = suicidalThoughtsdf.groupby(['age', 'year']).size().reset_index(name='count')
print(count_data)
print(suicidalThoughtsdf.head(10))

While there is an increase in the 26-34 age group from year to year.
The fluctuating values in the other age groups makes it difficult to determine a relationship between years for the other age groups.

In [ ]:
# making a sns graph specifically for comparing with suicidal thoughts
STsnsdf = coreTrendsAlldf.filter(['age', 'year'])
sns_columns = coreTrendsAlldf.filter(like='Sns_').columns
STsnsdf['sns_sum'] = coreTrendsAlldf[sns_columns].sum(axis=1)
# filter only those in the age group
STsnsdf = STsnsdf[STsnsdf['age'] >= 26]
STsnsdf = STsnsdf[STsnsdf['age'] <= 34]
print(STsnsdf.head(), STsnsdf.count())

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=STsnsdf, x='age', y='sns_sum', hue='year')
plt.title('Box Plot of sns_sum vs Age')
plt.xlabel('Age')
plt.ylabel('sns_sum')
plt.show()

Comparing the social media usage between the yearas for the 26-34 age group, there doesn't seem to be a consistent pattern in any of it connect it to suicidal thoughts.